In [68]:
from pyomo.environ import *
from pyomo.dae import *
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [86]:
def ABC_model(data):
    
    ca_meas = data['ca_meas']
    cb_meas = data['cb_meas']
    cc_meas = data['cc_meas']
    
    meas_t = list(ca_meas.keys())
       
    ca0 = 1.0
    cb0 = 0.0
    cc0 = 0.0
        
    m = ConcreteModel()
    
    m.k1 = Var(initialize = 0.5, bounds = (1e-4, 10))
    m.k2 = Var(initialize = 3.0, bounds = (1e-4, 10))
    
    m.time = ContinuousSet(bounds = (0.0, 5.0), initialize = meas_t)
    m.ca = Var(m.time, initialize = 0.5, bounds = (0, ca0))
    m.cb = Var(m.time, initialize = 0.5, bounds = (0, ca0))
    m.cc = Var(m.time, initialize = 0.5, bounds = (0, ca0))
    
    m.dca = DerivativeVar(m.ca, wrt = m.time)
    m.dcb = DerivativeVar(m.cb, wrt = m.time)
    m.dcc = DerivativeVar(m.cc, wrt = m.time)
    
    def _dcarate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dca[t] == -m.k1 * m.ca[t]
    m.dcarate = Constraint(m.time, rule = _dcarate)
    
    def _dcbrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcb[t] == m.k1 * m.ca[t] - m.k2 * m.cb[t]
    m.dcbrate = Constraint(m.time, rule = _dcbrate)
    
    def _dccrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcc[t] == m.k2 * m.cb[t]
    m.dccrate = Constraint(m.time, rule = _dccrate)
    
    def _initcon(m):
        yield m.ca[m.time.first()] == ca0
        yield m.cb[m.time.first()] == cb0
        yield m.cc[m.time.first()] == cc0
    m.initcon = ConstraintList(rule = _initcon)
    
    def ComputeFirstStageCost_rule(m):
        return 0
    m.FirstStageCost = Expression(rule=ComputeFirstStageCost_rule)

    def ComputeSecondStageCost_rule(m):
        return sum((m.ca[t] - ca_meas[t]) ** 2 + (m.cb[t] - cb_meas[t]) ** 2 
                   + (m.cc[t] - cc_meas[t]) ** 2 for t in meas_t) 
    m.SecondStageCost = Expression(rule=ComputeSecondStageCost_rule)

    
    def total_cost_rule(model):
        return model.FirstStageCost + model.SecondStageCost
    m.Total_Cost_Objective = Objective(rule=total_cost_rule, sense=minimize)
    
    disc = TransformationFactory('dae.collocation')
    disc.apply_to(m, nfe=20, ncp=4)
    
    return m

In [38]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

,t,ca,cb,cc
0,0.000,0.957,-0.031,-0.015
1,0.263,0.557,0.330,0.044
2,0.526,0.342,0.512,0.156
3,0.789,0.224,0.499,0.310
4,1.053,0.123,0.428,0.454


In [87]:
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]

In [88]:
theta_names = ['k1', 'k2']

In [89]:
pest = parmest.Estimator(ABC_model, data, theta_names, tee = True)

In [90]:
obj, theta = pest.theta_est()

Ipopt 3.11.1: max_iter=6000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:     2327
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      220

Total number of variables............................:      487
                     variables with only lower bounds:        0
                variables with lower and upper bou

  86  8.3146300e+00 1.89e-07 2.49e-01  -2.5 1.32e-05   4.3 1.00e+00 1.00e+00h  1
  87  8.3145181e+00 1.88e-07 2.49e-01  -2.5 4.93e-06   4.7 1.00e+00 1.00e+00h  1
  88  8.3141823e+00 1.87e-07 2.49e-01  -2.5 1.48e-05   4.2 1.00e+00 1.00e+00h  1
  89  8.3140565e+00 1.85e-07 2.49e-01  -2.5 5.55e-06   4.7 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  8.3136788e+00 1.84e-07 2.49e-01  -2.5 1.67e-05   4.2 1.00e+00 1.00e+00h  1
  91  8.3135372e+00 1.83e-07 2.49e-01  -2.5 6.24e-06   4.6 1.00e+00 1.00e+00h  1
  92  8.3131124e+00 1.82e-07 2.49e-01  -2.5 1.87e-05   4.1 1.00e+00 1.00e+00h  1
  93  8.3129530e+00 1.81e-07 2.49e-01  -2.5 7.02e-06   4.6 1.00e+00 1.00e+00h  1
  94  8.3124751e+00 1.80e-07 2.49e-01  -2.5 2.11e-05   4.1 1.00e+00 1.00e+00h  1
  95  8.3122959e+00 1.79e-07 2.49e-01  -2.5 7.90e-06   4.5 1.00e+00 1.00e+00h  1
  96  8.3117583e+00 1.78e-07 2.49e-01  -2.5 2.37e-05   4.0 1.00e+00 1.00e+00h  1
  97  8.3115568e+00 1.77e-07

 216  8.2768505e+00 1.17e-07 2.48e-01  -2.5 6.76e-06   4.6 1.00e+00 1.00e+00h  1
 217  8.2763909e+00 1.16e-07 2.48e-01  -2.5 2.03e-05   4.1 1.00e+00 1.00e+00h  1
 218  8.2762186e+00 1.16e-07 2.48e-01  -2.5 7.60e-06   4.5 1.00e+00 1.00e+00h  1
 219  8.2757016e+00 1.16e-07 2.48e-01  -2.5 2.28e-05   4.0 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 220  8.2755077e+00 1.15e-07 2.48e-01  -2.5 8.55e-06   4.5 1.00e+00 1.00e+00h  1
 221  8.2749261e+00 1.15e-07 2.48e-01  -2.5 2.57e-05   4.0 1.00e+00 1.00e+00h  1
 222  8.2747081e+00 1.15e-07 2.48e-01  -2.5 9.62e-06   4.4 1.00e+00 1.00e+00h  1
 223  8.2740539e+00 1.15e-07 2.48e-01  -2.5 2.89e-05   3.9 1.00e+00 1.00e+00h  1
 224  8.2738086e+00 1.14e-07 2.48e-01  -2.5 1.08e-05   4.4 1.00e+00 1.00e+00h  1
 225  8.2737166e+00 1.14e-07 2.48e-01  -2.5 4.06e-06   4.8 1.00e+00 1.00e+00h  1
 226  8.2734406e+00 1.14e-07 2.48e-01  -2.5 1.22e-05   4.3 1.00e+00 1.00e+00h  1
 227  8.2733371e+00 1.13e-07

 347  8.2380151e+00 9.05e-08 2.47e-01  -2.5 1.04e-05   4.4 1.00e+00 1.00e+00h  1
 348  8.2379266e+00 9.04e-08 2.47e-01  -2.5 3.91e-06   4.8 1.00e+00 1.00e+00h  1
 349  8.2376612e+00 9.02e-08 2.47e-01  -2.5 1.17e-05   4.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 350  8.2375617e+00 9.01e-08 2.47e-01  -2.5 4.40e-06   4.8 1.00e+00 1.00e+00h  1
 351  8.2372632e+00 8.99e-08 2.47e-01  -2.5 1.32e-05   4.3 1.00e+00 1.00e+00h  1
 352  8.2371512e+00 8.98e-08 2.47e-01  -2.5 4.94e-06   4.7 1.00e+00 1.00e+00h  1
 353  8.2368154e+00 8.97e-08 2.47e-01  -2.5 1.48e-05   4.2 1.00e+00 1.00e+00h  1
 354  8.2366894e+00 8.95e-08 2.47e-01  -2.5 5.56e-06   4.6 1.00e+00 1.00e+00h  1
 355  8.2363116e+00 8.94e-08 2.47e-01  -2.5 1.67e-05   4.2 1.00e+00 1.00e+00h  1
 356  8.2361700e+00 8.93e-08 2.47e-01  -2.5 6.26e-06   4.6 1.00e+00 1.00e+00h  1
 357  8.2357450e+00 8.91e-08 2.47e-01  -2.5 1.88e-05   4.1 1.00e+00 1.00e+00h  1
 358  8.2355856e+00 8.90e-08

 479  8.2000332e+00 7.60e-08 2.46e-01  -2.5 6.02e-06   4.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480  8.1996245e+00 7.60e-08 2.46e-01  -2.5 1.81e-05   4.1 1.00e+00 1.00e+00h  1
 481  8.1994713e+00 7.59e-08 2.46e-01  -2.5 6.77e-06   4.6 1.00e+00 1.00e+00h  1
 482  8.1990115e+00 7.58e-08 2.46e-01  -2.5 2.03e-05   4.1 1.00e+00 1.00e+00h  1
 483  8.1988392e+00 7.57e-08 2.46e-01  -2.5 7.62e-06   4.5 1.00e+00 1.00e+00h  1
 484  8.1983220e+00 7.56e-08 2.46e-01  -2.5 2.29e-05   4.0 1.00e+00 1.00e+00h  1
 485  8.1981281e+00 7.55e-08 2.46e-01  -2.5 8.57e-06   4.5 1.00e+00 1.00e+00h  1
 486  8.1975464e+00 7.54e-08 2.46e-01  -2.5 2.57e-05   4.0 1.00e+00 1.00e+00f  1
 487  8.1973282e+00 7.54e-08 2.46e-01  -2.5 9.64e-06   4.4 1.00e+00 1.00e+00h  1
 488  8.1966738e+00 7.53e-08 2.46e-01  -2.5 2.89e-05   3.9 1.00e+00 1.00e+00f  1
 489  8.1964285e+00 7.52e-08 2.46e-01  -2.5 1.08e-05   4.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr 

...

5398  7.0009572e+00 1.83e-08 2.15e-01  -2.5 3.54e-06   4.8 1.00e+00 1.00e+00h  1
5399  7.0007255e+00 1.83e-08 2.15e-01  -2.5 1.06e-05   4.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5400  7.0006386e+00 1.83e-08 2.15e-01  -2.5 3.99e-06   4.7 1.00e+00 1.00e+00h  1
5401  7.0003779e+00 1.83e-08 2.15e-01  -2.5 1.20e-05   4.3 1.00e+00 1.00e+00h  1
5402  7.0002801e+00 1.83e-08 2.15e-01  -2.5 4.49e-06   4.7 1.00e+00 1.00e+00h  1
5403  6.9999869e+00 1.83e-08 2.15e-01  -2.5 1.35e-05   4.2 1.00e+00 1.00e+00h  1
5404  6.9998769e+00 1.83e-08 2.15e-01  -2.5 5.05e-06   4.6 1.00e+00 1.00e+00h  1
5405  6.9995470e+00 1.83e-08 2.15e-01  -2.5 1.51e-05   4.2 1.00e+00 1.00e+00f  1
5406  6.9994233e+00 1.83e-08 2.15e-01  -2.5 5.68e-06   4.6 1.00e+00 1.00e+00h  1
5407  6.9990522e+00 1.83e-08 2.15e-01  -2.5 1.70e-05   4.1 1.00e+00 1.00e+00f  1
5408  6.9989131e+00 1.83e-08 2.15e-01  -2.5 6.39e-06   4.5 1.00e+00 1.00e+00h  1
5409  6.9984956e+00 1.83e-08

5530  6.9741530e+00 1.80e-08 2.14e-01  -2.5 1.64e-05   4.1 1.00e+00 1.00e+00f  1
5531  6.9740191e+00 1.80e-08 2.14e-01  -2.5 6.15e-06   4.5 1.00e+00 1.00e+00h  1
5532  6.9736173e+00 1.80e-08 2.14e-01  -2.5 1.85e-05   4.1 1.00e+00 1.00e+00f  1
5533  6.9734667e+00 1.80e-08 2.14e-01  -2.5 6.92e-06   4.5 1.00e+00 1.00e+00h  1
5534  6.9730148e+00 1.80e-08 2.14e-01  -2.5 2.08e-05   4.0 1.00e+00 1.00e+00f  1
5535  6.9728453e+00 1.80e-08 2.14e-01  -2.5 7.78e-06   4.4 1.00e+00 1.00e+00h  1
5536  6.9727817e+00 1.80e-08 2.14e-01  -2.5 2.92e-06   4.9 1.00e+00 1.00e+00h  1
5537  6.9725911e+00 1.80e-08 2.14e-01  -2.5 8.76e-06   4.4 1.00e+00 1.00e+00h  1
5538  6.9725196e+00 1.80e-08 2.14e-01  -2.5 3.28e-06   4.8 1.00e+00 1.00e+00h  1
5539  6.9723051e+00 1.80e-08 2.14e-01  -2.5 9.85e-06   4.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5540  6.9722247e+00 1.80e-08 2.14e-01  -2.5 3.69e-06   4.8 1.00e+00 1.00e+00h  1
5541  6.9719835e+00 1.80e-08

5663  6.9475110e+00 1.77e-08 2.14e-01  -2.5 3.56e-06   4.8 1.00e+00 1.00e+00h  1
5664  6.9472788e+00 1.77e-08 2.14e-01  -2.5 1.07e-05   4.3 1.00e+00 1.00e+00h  1
5665  6.9471917e+00 1.77e-08 2.14e-01  -2.5 4.00e-06   4.7 1.00e+00 1.00e+00h  1
5666  6.9469305e+00 1.77e-08 2.14e-01  -2.5 1.20e-05   4.3 1.00e+00 1.00e+00h  1
5667  6.9468326e+00 1.77e-08 2.14e-01  -2.5 4.50e-06   4.7 1.00e+00 1.00e+00h  1
5668  6.9465387e+00 1.77e-08 2.14e-01  -2.5 1.35e-05   4.2 1.00e+00 1.00e+00h  1
5669  6.9464286e+00 1.77e-08 2.14e-01  -2.5 5.07e-06   4.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5670  6.9460980e+00 1.77e-08 2.14e-01  -2.5 1.52e-05   4.1 1.00e+00 1.00e+00f  1
5671  6.9459741e+00 1.77e-08 2.14e-01  -2.5 5.70e-06   4.6 1.00e+00 1.00e+00h  1
5672  6.9456022e+00 1.77e-08 2.14e-01  -2.5 1.71e-05   4.1 1.00e+00 1.00e+00f  1
5673  6.9454628e+00 1.77e-08 2.14e-01  -2.5 6.41e-06   4.5 1.00e+00 1.00e+00h  1
5674  6.9450445e+00 1.77e-08

5794  6.9211447e+00 9.08e-09 2.18e-01  -3.8 5.61e-06   4.6 1.00e+00 1.00e+00h  1
5795  6.9207890e+00 9.08e-09 2.18e-01  -3.8 1.68e-05   4.1 1.00e+00 1.00e+00f  1
5796  6.9206556e+00 9.07e-09 2.18e-01  -3.8 6.31e-06   4.5 1.00e+00 1.00e+00h  1
5797  6.9202554e+00 9.07e-09 2.18e-01  -3.8 1.89e-05   4.1 1.00e+00 1.00e+00f  1
5798  6.9201054e+00 9.07e-09 2.18e-01  -3.8 7.10e-06   4.5 1.00e+00 1.00e+00h  1
5799  6.9196552e+00 9.07e-09 2.18e-01  -3.8 2.13e-05   4.0 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5800  6.9194864e+00 9.07e-09 2.18e-01  -3.8 7.99e-06   4.4 1.00e+00 1.00e+00h  1
5801  6.9194231e+00 9.07e-09 2.18e-01  -3.8 3.00e-06   4.9 1.00e+00 1.00e+00h  1
5802  6.9192332e+00 9.07e-09 2.18e-01  -3.8 8.99e-06   4.4 1.00e+00 1.00e+00h  1
5803  6.9191620e+00 9.07e-09 2.18e-01  -3.8 3.37e-06   4.8 1.00e+00 1.00e+00h  1
5804  6.9189484e+00 9.07e-09 2.18e-01  -3.8 1.01e-05   4.3 1.00e+00 1.00e+00h  1
5805  6.9188683e+00 9.07e-09

5926  6.8945326e+00 9.07e-09 2.17e-01  -3.8 3.25e-06   4.8 1.00e+00 1.00e+00h  1
5927  6.8943270e+00 9.07e-09 2.17e-01  -3.8 9.74e-06   4.3 1.00e+00 1.00e+00h  1
5928  6.8942499e+00 9.07e-09 2.17e-01  -3.8 3.65e-06   4.8 1.00e+00 1.00e+00h  1
5929  6.8940186e+00 9.07e-09 2.17e-01  -3.8 1.10e-05   4.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5930  6.8939318e+00 9.07e-09 2.17e-01  -3.8 4.11e-06   4.7 1.00e+00 1.00e+00h  1
5931  6.8936716e+00 9.07e-09 2.17e-01  -3.8 1.23e-05   4.2 1.00e+00 1.00e+00f  1
5932  6.8935740e+00 9.07e-09 2.17e-01  -3.8 4.62e-06   4.7 1.00e+00 1.00e+00h  1
5933  6.8932813e+00 9.07e-09 2.17e-01  -3.8 1.39e-05   4.2 1.00e+00 1.00e+00f  1
5934  6.8931715e+00 9.07e-09 2.17e-01  -3.8 5.20e-06   4.6 1.00e+00 1.00e+00h  1
5935  6.8928422e+00 9.07e-09 2.17e-01  -3.8 1.56e-05   4.1 1.00e+00 1.00e+00f  1
5936  6.8927188e+00 9.07e-09 2.17e-01  -3.8 5.85e-06   4.6 1.00e+00 1.00e+00h  1
5937  6.8923483e+00 9.07e-09